# 🚀 Train Your First Custom Wake Word with Nanowakeword!

Welcome to the official tutorial for **Nanowakeword**! 

In this notebook, we will guide you through the entire process of training a high-performance, custom wake word model from scratch. You don't need any pre-existing data—we will download everything we need and let Nanowakeword's intelligent engine do the heavy lifting.

**Our goal:** Go from zero to a ready-to-use wake word model in just a few simple steps. Let's get started!

**Installation**

In [ ]:
# @title Step 1: Install Nanowakeword
# We install the full [train] package to get all the necessary dependencies.

! pip install --no-cache-dir "nanowakeword[train]==1.3.0"
! pip install piper-tts

print("Installation complete!")

## Step 2: Prepare the Dataset

A great model starts with great data. For this tutorial, we will:
1.  **Download** open-source noise and Room Impulse Response (RIR) datasets.
2.  **Organize** all your project files within a clean, well-structured folder hierarchy for better clarity and maintainability.

In [1]:
# @title Step 2: Download & Prepare the SonicWeave-v1 Dataset

import os
from pathlib import Path
import subprocess
import shutil

# --- Configuration ---
DATASET_REPO_URL = "https://huggingface.co/datasets/arcosoph/SonicWeave-v1"
DATA_DIR = Path("./nanowakeword_data")

# --- Define Final Paths ---
noise_dir = DATA_DIR / "Noise"
rir_dir = DATA_DIR / "Rir"
positive_dir = DATA_DIR / "positive_wakeword"
negative_dir = DATA_DIR / "negative_speech"

# --- Main Logic ---
print("The dataset is downloading. This may take a moment...")

# Download only if the dataset folders are not already created.
if not noise_dir.exists() or not rir_dir.exists() or not any(noise_dir.iterdir()):
    
    # Clone the repository to a temporary location
    temp_clone_dir = DATA_DIR / "temp_repo"
    
    print(f"Downloading the Starter Dataset from {DATASET_REPO_URL}...")
    
    # --depth 1 only downloads the latest commit, which is much faster      
    try:
        subprocess.run(
            ["git", "clone", "--depth", "1", DATASET_REPO_URL, str(temp_clone_dir)],
            check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL # Hide unnecessary log messages
        )
    except subprocess.CalledProcessError as e:
        print(f"Error: Failed to clone the dataset repository. Please check the URL.")
        print(f"Git command failed with error: {e}")
    else:
        print("Organizing dataset files...")

        # Move the noise and rir folders from the cloned repository to the correct location
        try:

            # Move only required folders
            for folder_name in ["Noise", "Rir"]:
                src = temp_clone_dir / folder_name
                dst = DATA_DIR / folder_name
                if src.exists():
                    shutil.move(str(src), str(dst))

            # Delete temp_repo, ignore errors if some files are locked
            shutil.rmtree(temp_clone_dir, ignore_errors=True)
                        
            print("\nSonicWeave-v1 Dataset is ready!")
        except FileNotFoundError:
            print("Error: 'noise' or 'rir' folder not found inside the cloned repository.")
        except Exception as e:
            print(f"Error organizing files: {e}")
else:
    print("SonicWeave-v1 Dataset already found.")

The dataset is downloading. This may take a moment...
SonicWeave-v1 Dataset already found.


**Generate Wake Word Samples**

## Step 3: Configure and Train the Model

Now for the fun part! We will create a `config.yaml` file and then run the Nanowakeword training command.

In [ ]:
# This section is for our README file.

# The Magic: Reading from URL or showing a dropdown
import ipywidgets as widgets
from google.colab import output

# Function to get query parameters from the URL
def get_query_param(param):
    js_code = f"return new URL(window.location.href).searchParams.get('{param}');"
    result = output.eval_js(js_code)
    return result if result else None

# Get model_type from URL, or default to 'dnn'
url_model_type = get_query_param('model_type')
default_model = url_model_type if url_model_type in ["dnn", "lstm", "gru", "cnn", "rnn"] else "dnn"

# Create the interactive dropdown widget
model_type_dropdown = widgets.Dropdown(
    options=['dnn', 'lstm', 'gru', 'cnn', 'rnn'],
    value=default_model,
    description='Model Type:',
    disabled=False,
)

# Display the dropdown to the user
display(model_type_dropdown)

# Later, in the config cell, you will use the selected value
# selected_model_type = model_type_dropdown.value

**Configuration, Generate Wake Word Samples and Training**

In [2]:
# @title Step 3.1: Create the Configuration File
import yaml

# Try to safely get model type, fallback to "dnn" if dropdown not defined
try:
    selected_model = model_type_dropdown.value
except NameError:
    selected_model = "dnn"

config_dict = {
    # Data Paths (pointing to our newly created folders)
    "positive_data_path": str(positive_dir),
    "negative_data_path": str(negative_dir),
    "background_paths": [str(noise_dir)],
    "rir_paths": [str(rir_dir)],
    # Model Output
    "output_dir": "./trained_models",
    "model_name": "hey_computer_v1",
    # your wakeword
    "target_phrase" : ["arcosoph"],  
    # How many synthetic data sets do you want to generate?
    'generate_positive_samples': 1,      # postive
    'generate_negative_samples': 2500,      # negative

    # 'augmentation_settings': {
    #     'BackgroundNoise': 0.75
    # },
    "steps": 20000,
    "layer_size": 256,
    "n_blocks": 3,
    'generate_clips': True,
    'transform_clips': True,
    'train_model': True,
    'overwrite': True,
    'debug_mode': True,
    # Model Type
    "model_type": selected_model # DNN offers faster training, while LSTM and other architectures provide greater robustness.
    
    # You can provide other parameters if you want...
}

# Write the config to a YAML file
config_path = "./config.yaml"
with open(config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False)

print(f"✅ Configuration file saved to {config_path}")

✅ Configuration file saved to ./config.yaml


**Run Training!**

In [3]:
# @title Step 3.2: Run the Magic Command! 🚀
# This command will do everything: augment data, extract features, and train the model.
# It might take some time depending on the hardware (especially on a CPU).

from nanowakeword.trainer import train 

args_list = [
    '--config_path', f'{config_path}',
]

print("Starting NanoWakeWord training...")

try:
    train(args_list)
    print("\n\nCONGRATULATIONS! (✿◕‿◕✿)")
    print("Your custom wake word model has been successfully trained!")

except Exception as e:
    print(f"\nAn error occurred during training: {e}")

f:\testnano\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\testnano\.venv\Lib\site-packages\pronouncing\__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream


Starting NanoWakeWord training...


  _   _               __          __   _     __          __           _ 
 | \ | |              \ \        / /  | |    \ \        / /          | |
 |  \| | __ _ _ __   __\ \  /\  / /_ _| | ____\ \  /\  / /__  _ __ __| |
 | . ` |/ _` | '_ \ / _ \ \/  \/ / _` | |/ / _ \ \/  \/ / _ \| '__/ _` |
 | |\  | (_| | | | | (_) \  /\  / (_| |   <  __/\  /\  / (_) | | | (_| |
 |_| \_|\__,_|_| |_|\___/ \/  \/ \__,_|_|\_\___| \/  \/ \___/|_|  \__,_|

STEP 1: Verifying and Preprocessing Data Directories

====================================================

...

INFO: Autotuning optimal clip duration...

INFO: Optimal clip duration autotuned to: 32000 samples (2.00 seconds).

STEP 3: Computing Nanowakeword features for generated samples

=============================================================


An error occurred during training: AddBackgroundNoise.__init__() got an unexpected keyword argument 'sounds'


## What's Next?

You have successfully trained your own custom wake word model!

You can now download the `.onnx` file from the `trained_models` directory (check the file browser on the left) and use it in your own applications.

For more advanced topics, such as using your own datasets or fine-tuning the configuration, please check out our full documentation on **[GitHub](https://github.com/arcosoph/nanowakeword)**.

---
## Step 4: Save Your Model to Google Drive

The final step is to save your trained model and performance graph to a safe and accessible place. Instead of a slow direct download, we will save the files directly to your Google Drive. This process is almost instantaneous.

Run the cells below to:
1.  Connect your Google Drive account.
2.  Copy all the trained files into a new folder named `nanowakeword_models` in your Drive.

In [ ]:
# @title Step 4.1: Connect to Google Drive
# This will ask for your permission to access your Google Drive.

from google.colab import drive
import os

try:
    drive.mount('/content/drive')
    print("\nGoogle Drive connected successfully!")
except Exception as e:
    print(f"An error occurred while connecting to Google Drive: {e}")

In [ ]:
# @title Step 4.2: Copy Final Model and Artifacts to Google Drive 📂

import os
import shutil

# --- Configuration ---
# Get model_name and output_dir from the config_dict defined earlier
model_name = config_dict.get("model_name", "my_model")
output_dir = config_dict.get("output_dir", "./trained_models")

# --- Source and Destination Paths ---
# The source project directory containing all generated files
source_project_dir = os.path.join(output_dir, model_name)

# The destination folder in your Google Drive
drive_destination_dir = f"./nanowakeword_models/{model_name}"

# --- Start Copy Process ---
print("Starting the process to copy trained files to Google Drive...")

# Check if the source directory exists
if not os.path.exists(source_project_dir):
    print(f"\n❌ ERROR: Source directory not found at '{source_project_dir}'")
    print("This indicates that the training process did not create the expected output folder.")
    print("Please ensure the training step completed successfully before running this cell.")
else:
    # If an old folder exists in Drive, remove it to ensure a clean copy
    if os.path.exists(drive_destination_dir):
        print(f"🔄 Found an existing folder in Drive. Removing it for a fresh copy: '{drive_destination_dir}'")
        shutil.rmtree(drive_destination_dir)

    # --- Copy the entire project folder ---
    # This is much simpler and more reliable than copying individual files.
    # It preserves the professional directory structure.
    try:
        shutil.copytree(source_project_dir, drive_destination_dir)
        
        print("\n" + "="*50)
        print("✅ SUCCESS! All files have been saved to your Google Drive.")
        print("="*50)
        print(f"\nYour complete project, including the model and performance graphs, can be found in:")
        print(f"➡️ '{drive_destination_dir}'")
        
        # Optional: List the contents of the new folder in Drive for verification
        print("\nContents of the saved folder:")
        for root, dirs, files in os.walk(drive_destination_dir):
            level = root.replace(drive_destination_dir, '').count(os.sep)
            indent = ' ' * 4 * (level)
            print(f"{indent}{os.path.basename(root)}/")
            sub_indent = ' ' * 4 * (level + 1)
            for f in files:
                print(f"{sub_indent}{f}")

    except Exception as e:
        print(f"\n❌ ERROR: An unexpected error occurred during the copy process.")
        print(f"Details: {e}")